# Model Evauation
Classification models need the following metrics

Cross Validation
- Accuracy
- error

ROC Curve
Steps
- load data
- featurize
- choose model
- pass data and HP's
- run CV
- ROC possible?
- tabulate results
- report

In [1]:
#### Standard Libraries ####
import os
from pprint import pprint
import numpy as np
import pandas as pd
import multiprocessing as mp
from functools import partial
from itertools import product
import timeit
import uuid

#### third-party Libraries ####
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier as SRC
from lolopy.learners import RandomForestClassifier as LRC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#### Local Libraries ####
from utils import (Result, run_k_folds, 
                   report_column_labels,
                   compile_data)
from data_manager import DataManager
from featurizer import Featurizer

In [75]:
# configuration
np.random.seed(8)
load_path = os.path.join('data','training_data.csv')
save_path = os.path.join('results','baseline_fixed_sample.csv')
mp_api_key = '7n6DwPUQ5cf8ZTWO'
oversample = False
data_ramp = True
feature_set = ['standard']

## Load data
This object should handle data load and sampling.

Should be able to do the following:
1. load and sample 
2. Run a data ramp

In [58]:
# Load Data
dm = DataManager(load_path, save_path)
dm.load()

'Loaded 2572 records.'


In [59]:
# Sample data
if not data_ramp:
    dm.sample_data(100)
    #dm.data.to_csv('data/sample_07_19_19.csv', index=False)

In [61]:
# Format and careate composition objects
#dm.compute_formula()
dm.to_binary_classes()
dm.get_pymatgen_composition()
dm.remove_noble_gasses()
dm.remove_features()

In [64]:
if data_ramp:
    dm.data = dm.data.sample(frac=1).reset_index(drop=True)

In [66]:
f = Featurizer(feature_set, mp_api_key)

In [67]:
dm.featurized_data = f.featurize(dm.data)

In [68]:
dm.groups = dm.data['group']

In [69]:
# Set training labels
dm.outputs = dm.data['stable']

In [70]:
k_folds = partial(run_k_folds, inputs=dm.featurized_data, outputs=dm.outputs, groups=dm.groups, sampling=oversample, ramp=data_ramp, splits=5)

In [76]:
# set models
models = [GaussianNB(), SVC(), SRC(), LogisticRegression(), DummyClassifier(strategy="most_frequent")]
#models = [SRC()]

In [77]:
%%capture
pool = mp.Pool(processes=mp.cpu_count())

In [78]:
%%capture
start_time = timeit.default_timer()
results = pool.map(k_folds, models)    
elapsed = timeit.default_timer() - start_time

In [79]:
compiled = []
for result in results:
    compiled.extend(compile_data(result))
res_df = pd.DataFrame(compiled, columns=report_column_labels)
res_df.to_csv(save_path)
res_df

,type,data_size,accuracy,accuracy_std,f1,f1_std,recall,recall_std,precision,precision_std
0,GaussianNB,4114,0.770057,0.019601,0.392930,0.039705,0.595923,0.047151,0.294117,0.037515
1,GaussianNB,8228,0.745751,0.025916,0.389440,0.026976,0.643349,0.033973,0.280196,0.026962
2,GaussianNB,12342,0.749066,0.014398,0.389991,0.031893,0.639151,0.041356,0.281067,0.027180
3,GaussianNB,16456,0.743926,0.015175,0.389613,0.028700,0.651367,0.063375,0.278905,0.023668
4,GaussianNB,20570,0.748274,0.014339,0.391613,0.021730,0.648737,0.049388,0.280810,0.016082
5,SVC,4114,0.875064,0.011776,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,SVC,8228,0.874091,0.011641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,SVC,12342,0.874333,0.009260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,SVC,16456,0.873605,0.013831,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,SVC,20570,0.875061,0.002801,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Single model test

In [18]:
import numpy
import pandas
from sklearn.utils import resample
from collections import Counter

def local_oversample(train, outputs):
    majority_class, minority_class = Counter(outputs[train]).most_common()
    minority = train[numpy.where(outputs[train] == minority_class[0])[0]]
    majority = train[numpy.where(outputs[train] == majority_class[0])[0]]
    minority_upsampled = resample(minority,
                              replace=True,
                              n_samples=len(majority),
                              random_state=8)
    t = numpy.append(majority, minority_upsampled)
    pprint(len(t))
    return (t, minority, majority, minority_upsampled)

In [19]:
model = SRC()
ovs = False

In [20]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import KFold, GroupKFold, LeavePGroupsOut, LeaveOneGroupOut
inputs = dm.featurized_data
outputs = dm.outputs
groups = dm.groups
gkf = GroupKFold(n_splits=10)
lpgo = LeavePGroupsOut(n_groups=1)
logo = LeaveOneGroupOut()
splits = list(gkf.split(inputs, outputs, groups=groups.values))
#splits = list(logo.split(inputs, outputs, groups))

In [21]:
train, test = splits[3]

In [22]:
%%capture
# Place all data into df's for easy analysis
train_df = pandas.DataFrame(dm.data.ix[train, ['formula','group','stable']])
test_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])

In [23]:
if ovs:
    train, minority, majority, minority_upsampled = local_oversample(train, outputs)

In [24]:
# Run the ML
model.fit(inputs[train], outputs[train])
prediction = model.predict(inputs[test])
acc = accuracy_score(outputs[test], prediction)
f1s = [precision_recall_fscore_support(
    outputs[test], prediction, labels=[0,1], average='binary')]

/home/mdavidson/.conda/envs/tecca/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



In [31]:
len(train)

18513

In [26]:
%%capture
# Place all data into df's for easy analysis
train_df = pandas.DataFrame(dm.data.ix[train, ['formula','group','stable']])
test_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])
pred_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])
pred_df['prediction'] = prediction
acc_df = pandas.DataFrame([acc], columns=['accuracy'])

f1_df = pandas.DataFrame(
        f1s, columns=['precision', 'recall', 'f1', 'support'])
ov_df = pd.DataFrame([train_df['group'].isin(test_df['group']).any()], columns=['overlap'])

try:
    lab_df = pandas.DataFrame([pred_df['prediction'].value_counts().values], columns=['pred_0','pred_1'])
except:
    lab_df = pandas.DataFrame([pred_df['prediction'].value_counts().values], columns=['pred_0'])
if ovs:
    sample_df = pandas.DataFrame([train_df['stable'].value_counts().values], columns=['smpl_0','smpl_1'])
else:
    sample_df = pandas.DataFrame([], columns=['sample'])
# Get the accuracy for each label
#ac_lab_df = pandas.DataFrame([test_df['stable'].value_counts().values], columns=['acc_0','acc_1'])

In [27]:
#res = f1_df.join(acc_df).join(ov_df).join(lab_df).join(ac_lab_df).join(sample_df)

In [28]:
res = f1_df.join(acc_df).join(ov_df).join(lab_df).join(sample_df)

In [29]:
#res.to_csv('results/final_modle.csv', index=False)

In [30]:
res

,precision,recall,f1,support,accuracy,overlap,pred_0,pred_1,sample
0,0.715232,0.413793,0.524272,None,0.904716,False,1906,151,NaN


In [255]:
import pickle
#pickle.dump(model, open('rfc.sav', 'wb+'))